## Introduction
### Background
According to the Migration Policy Institute, more than 44,7 million immigrants lived in the United States in 2018. Then, according to Census.gov, 1.21 million people migrated to the US in 2018. Immigration has been a major source of population growth and cultural change throughout much of U.S. history. And one of the questions new arrivals are going to ask is where to settle down to start a new chapter in their life. We will try to answer this question on the example of Los Angeles.

### Problem
In this case, we are adopting machine learning tools to assist immigrants to find apartments with optimum conditions. The question we are currently solving is: how could we provide support to immigrants in finding suitable apartments in Los Angeles(further - LA)?
To solve this problem, we are going to cluster LA neighborhoods in order to recommend venues with the median renting price amount where immigrants can find suitable apartments. We will recommend neighborhoods according to essential facilities and services surrounding such venues i.e. schools, stores, restaurants, hospitals, and entertainments.


## Data
Data about Los Angeles(further - LA) neighborhoods and rent price paid in these neighborhoods was taken from USC Price Center for Social Innovation    (https://usc.data.socrata.com/Los-Angeles/Rent-Price-LA-/4a97-v5tx). The dataset comprises LA Neighborhoods, Median Rent Price Amount in LA Neighborhoods.

To discover and choose affordable and convenient locations across different neighborhoods according to the presence of essential facilities and services, we will access data through FourSquare API interface. By merging data frames on LA neighborhoods and the median rent price amount from USC price center and data on essential facilities and services surrounding these neighborhoods from FourSquare API interface, we will be able to recommend neighborhoods with affordable renting apartments.



## Methodology
#### The Methodology section is consist of  four following stages:

 1. Data Collection 
 2. Data Exploration and Understanding
 3. Data preparation and preprocessing 
 4. Modeling

## Importing Libraries
## Data Collection from USC Price Center for Social Innovation

In [1]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

In [2]:
!pip install geopy
from geopy.geocoders import Nominatim

In [49]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

In [4]:
df = pd.read_csv('https://usc.data.socrata.com/api/views/4a97-v5tx/rows.csv')

## Data Exploration and Understanding

In [5]:
df.head()

,Policy Area,Dataset,Variable,Year,Amount,Tract,Tract Number,Neighborhood,GEOID,Location,Row ID,Date
0,Housing & Real Estate,Rent Price,Median Rent Price,2015,1115.0,"Census Tract 9200.23, Los Angeles County, Cali...",920023,Santa Clarita,1400000US06037920023,"(34.4214975, -118.4905135)",Median_Rent_Price_2015_1400000US06037920023,01/01/2015
1,Housing & Real Estate,Rent Price,Median Rent Price,2015,1038.0,"Census Tract 5344.04, Los Angeles County, Cali...",534404,Cudahy,1400000US06037534404,"(33.966232, -118.190553)",Median_Rent_Price_2015_1400000US06037534404,01/01/2015
2,Housing & Real Estate,Rent Price,Median Rent Price,2015,1031.0,"Census Tract 5336.01, Los Angeles County, Cali...",533601,Bell,1400000US06037533601,"(33.9803325, -118.194495)",Median_Rent_Price_2015_1400000US06037533601,01/01/2015
3,Housing & Real Estate,Rent Price,Median Rent Price,2015,1362.0,"Census Tract 4053.02, Los Angeles County, Cali...",405302,West Covina,1400000US06037405302,"(34.0767645, -117.946434)",Median_Rent_Price_2015_1400000US06037405302,01/01/2015
4,Housing & Real Estate,Rent Price,Median Rent Price,2015,919.0,"Census Tract 2042, Los Angeles County, California",204200,Boyle Heights,1400000US06037204200,"(34.043615, -118.205084)",Median_Rent_Price_2015_1400000US06037204200,01/01/2015


In [6]:
df.shape

(16390, 12)

In [7]:
df.dtypes

Policy Area      object
Dataset          object
Variable         object
Year              int64
Amount          float64
Tract            object
Tract Number      int64
Neighborhood     object
GEOID            object
Location         object
Row ID           object
Date             object
dtype: object

Dataset consist of 16390 rows and 12 columns. Now we can clean dataset and extract necessary columns 

##  Dataset preparation  and preprocessing
#### We prepare our dataset for the modeling process, by performing the following steps:
 1. Extract columns related to Neighborhoods, Median Rent Price and Location
 2. Rename the column name
 3. Select neighborhoods which median rent price fit the price range 
 4. Import venues around neighborhoods
 5. Join the data to find the coordinates of suitable locations 
 6. Plot recommended locations on LA map

#### Formating collected data

In [8]:
df = df[['Amount','Neighborhood', 'Location']]
df = df.rename(columns={'Amount':'Median Rent Price'})

In [9]:
df[['Latitude', 'Longitude']] = df['Location'].str.split(',', n=1, expand=True)

In [10]:
df = df.drop(['Location'], axis=1)

#### Preparing coordinates of neighboords

In [11]:
df['Latitude']= df['Latitude'].str.replace('(', '')
df['Longitude']= df['Longitude'].str.replace(')', '')
df.head()

,Median Rent Price,Neighborhood,Latitude,Longitude
0,1115.0,Santa Clarita,34.4214975,-118.4905135
1,1038.0,Cudahy,33.966232,-118.190553
2,1031.0,Bell,33.9803325,-118.194495
3,1362.0,West Covina,34.0767645,-117.946434
4,919.0,Boyle Heights,34.043615,-118.205084


In [12]:
df[['Latitude', 'Longitude']] = df[['Latitude', 'Longitude']].astype('float64')
df.dtypes

Median Rent Price    float64
Neighborhood          object
Latitude             float64
Longitude            float64
dtype: object

#### Finding Median Rent Price among all Neighborhoods to narrow dataset

In [13]:
df.describe()

,Median Rent Price,Latitude,Longitude
count,16055.000000,16390.000000,16390.000000
mean,1316.910308,34.065633,-118.251864
std,398.110637,0.179466,0.190607
min,100.000000,33.343563,-118.855281
25%,1031.000000,33.957001,-118.380005
50%,1213.000000,34.054677,-118.269599
75%,1520.000000,34.152711,-118.136926
max,3500.000000,34.792994,-117.708132


In [14]:
df1 = df[df['Median Rent Price'] >= 1315]
la_neighborhoods = df1[df1['Median Rent Price'] <= 1350]

#### Preparing final dataset

In [15]:
la_neighborhoods.head()

,Median Rent Price,Neighborhood,Latitude,Longitude
120,1348.0,Long Beach,33.822661,-118.176187
128,1330.0,Lancaster,34.711343,-118.157119
183,1333.0,Quartz Hill,34.638321,-118.221842
223,1336.0,Torrance,33.877806,-118.330794
243,1343.0,Sun Valley,34.211160,-118.371144


In [16]:
la_neighborhoods['Neighborhood'].value_counts()

Baldwin Park                             15
Pacoima                                  14
Long Beach                               14
Palmdale                                 12
Glendale                                 12
Alhambra                                 11
Downey                                   11
Monrovia                                 10
Pasadena                                 10
Monterey Park                             9
Azusa                                     8
Paramount                                 8
Burbank                                   8
Lakewood                                  8
Palms                                     8
Torrance                                  8
Bellflower                                8
West Covina                               8
Whittier                                  7
South Whittier                            7
Norwalk                                   7
Sun Valley                                6
Panorama City                   

In [17]:
la_neighborhoods.shape

(493, 4)

#### Getting the latitude and longitude values of Los Angeles

In [18]:
address = 'Los Angeles, CA'

geolocator = Nominatim(user_agent="la_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Los Angeles are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Los Angeles are 34.0536909, -118.2427666.


#### Creating a map of Los Angeles with neighborhoods superimposed on top

In [19]:
map_los_angeles = folium.Map(location=[latitude, longitude], zoom_start=11)


for lat, lng, label in zip(la_neighborhoods['Latitude'], la_neighborhoods['Longitude'], la_neighborhoods['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_los_angeles)  
    
map_los_angeles

#### Defining Foursquare Credentials and Version

In [20]:
CLIENT_ID = '41CSPZAYRM2LSTNHXVE0OVO5OJG1EGNFRNKURBFUZAK0YB1V' 
CLIENT_SECRET = 'ZFD1L1Z1VHJA4HUF2NISSU0PR2UMIA5KK3XC1WJRB5KKRBOZ' 
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 41CSPZAYRM2LSTNHXVE0OVO5OJG1EGNFRNKURBFUZAK0YB1V
CLIENT_SECRET:ZFD1L1Z1VHJA4HUF2NISSU0PR2UMIA5KK3XC1WJRB5KKRBOZ


# Modeling

#### Exploring Neighborhoods in Los Angeles

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=200, LIMIT=50):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Creating Los Angeles Neighborhoods Venues dataframe (la_neighborhoods_venues)

In [23]:
la_neighborhoods_venues = getNearbyVenues(names=la_neighborhoods['Neighborhood'],
                                   latitudes=la_neighborhoods['Latitude'],
                                   longitudes=la_neighborhoods['Longitude']
                                  )

Long Beach
Lancaster
Quartz Hill
Torrance
Sun Valley
Pacoima
Santa Clarita
Baldwin Park
Artesia
Long Beach
Mid-Wilshire
Pacoima
Alhambra
Harbor City
Baldwin Park
Whittier
Fairfax
Industry
Rancho Park
South Whittier
San Pedro
Compton
Echo Park
Sun Valley
Downey
Norwalk
West Puente Valley
Elizabeth Lake
South San Jose Hills
Paramount
Pomona
El Monte
Sylmar
Santa Fe Springs
Sierra Madre
Palmdale
Inglewood
Azusa
Long Beach
Long Beach
Arcadia
Downey
San Pedro
Mar Vista
San Gabriel
South El Monte
Porter Ranch
Manchester Square
Bellflower
Winnetka
Norwalk
Pico Rivera
Alhambra
Paramount
Monrovia
South El Monte
El Monte
Montebello
Alhambra
West Covina
Alhambra
South San Jose Hills
Whittier
Montecito Heights
Palmdale
Alhambra
Palmdale
Monrovia
Pico Rivera
San Pasqual
Whittier Narrows
Claremont
Glendale
Hawaiian Gardens
Burbank
Highland Park
Burbank
Pomona
Downey
Bellflower
Downey
South Whittier
Lakewood
Sherman Oaks
Inglewood
Green Meadows
West Covina
Azusa
Pasadena
Alhambra
Silver Lake
Artesia


In [51]:
la_neighborhoods_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Quartz Hill,34.638321,-118.221842,The Brandwave,34.637017,-118.222812,Print Shop
1,Sun Valley,34.211160,-118.371144,Active Key Insurance,34.210134,-118.370863,Office
2,Sun Valley,34.211160,-118.371144,Vineland / Stagg,34.212018,-118.370521,Bus Station
3,Artesia,33.859614,-118.079953,Sharetea,33.857890,-118.080290,Bubble Tea Shop
4,Artesia,33.859614,-118.079953,Rainbow Donuts,33.858746,-118.081146,Bakery


#### Amount of venues were returned for each neighborhood

In [27]:
la_neighborhoods.groupby('Neighborhood').count()

,Median Rent Price,Latitude,Longitude
Neighborhood,,,
Alhambra,11,11,11
Altadena,1,1,1
Arcadia,3,3,3
Arleta,2,2,2
Artesia,6,6,6
Atwater Village,2,2,2
Avocado Heights,2,2,2
Azusa,8,8,8
Baldwin Park,15,15,15


#### Analyzing Each Neighborhood

In [25]:
la_neighborhoods_onehot = pd.get_dummies(la_neighborhoods_venues[['Venue Category']], prefix="", prefix_sep="")

la_neighborhoods_onehot['Neighborhood'] = la_neighborhoods_venues['Neighborhood'] 

fixed_columns = [la_neighborhoods_onehot.columns[-1]] + list(la_neighborhoods_onehot.columns[:-1])
la_neighborhoods_onehot = la_neighborhoods_onehot[fixed_columns]

la_neighborhoods_onehot.head()

,Zoo Exhibit,ATM,Accessories Store,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Garden,Big Box Store,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Bus Station,Bus Stop,Business Service,Café,Cajun / Creole Restaurant,Campground,Candy Store,Cantonese Restaurant,Carpet Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Ethiopian Restaurant,Event Space,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,Food & Drink Shop,Food Truck,Football Stadium,French Restaurant,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Lawyer,Lighthouse,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Office,Optical Shop,Organic Grocery,Other Repair Shop,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pop-Up Shop,Print Shop,Pub,Public Art,Ramen Restaurant,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Snack Place,Sorority House,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Tattoo Parlor,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toy / Game Store,Trail,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Quartz Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Sun Valley,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [26]:
la_neighborhoods_onehot.shape

(1111, 224)

#### Frequency of occurrence of each category of venues in Neighborhoods

In [29]:
la_neighborhoods_grouped = la_neighborhoods_onehot.groupby('Neighborhood').mean().reset_index()
la_neighborhoods_grouped.head()

,Neighborhood,Zoo Exhibit,ATM,Accessories Store,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Garden,Big Box Store,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Bus Station,Bus Stop,Business Service,Café,Cajun / Creole Restaurant,Campground,Candy Store,Cantonese Restaurant,Carpet Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Ethiopian Restaurant,Event Space,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,Food & Drink Shop,Food Truck,Football Stadium,French Restaurant,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Lawyer,Lighthouse,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Nightlife Spot,Office,Optical Shop,Organic Grocery,Other Repair Shop,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pop-Up Shop,Print Shop,Pub,Public Art,Ramen Restaurant,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Snack Place,Sorority House,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Tattoo Parlor,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toy / Game Store,Trail,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Alhambra,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.076923,0.0,0.0,0.000000,0.0,0.000000,0.0,0.153846,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.153846,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.076923,0.0,0.0,0.0,0.0,0.0,0.153846,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [32]:
la_neighborhoods_grouped.shape

(97, 224)

#### Finding Top 10 venues in each neighborhood

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = la_neighborhoods_grouped['Neighborhood']

for ind in np.arange(la_neighborhoods_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(la_neighborhoods_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alhambra,Hotel,Burger Joint,Pharmacy,Video Store,Park,Nightclub,Shop & Service,Bowling Alley,Discount Store,Fast Food Restaurant
1,Altadena,Intersection,Yoga Studio,Event Space,Furniture / Home Store,French Restaurant,Football Stadium,Food Truck,Food & Drink Shop,Food,Flea Market
2,Arcadia,Garden,Sculpture Garden,Cuban Restaurant,Furniture / Home Store,French Restaurant,Football Stadium,Food Truck,Food & Drink Shop,Food,Flea Market
3,Artesia,Indian Restaurant,Bubble Tea Shop,Playground,Chinese Restaurant,Ice Cream Shop,Vietnamese Restaurant,Sandwich Place,Dessert Shop,Sushi Restaurant,Bakery
4,Atwater Village,Pizza Place,Liquor Store,Yoga Studio,Ethiopian Restaurant,French Restaurant,Football Stadium,Food Truck,Food & Drink Shop,Food,Flea Market


### Clustering Neighborhoods

#### Running k-means to cluster the neighborhood into 5 clusters

In [38]:
kclusters = 5

la_neighborhoods_grouped_clustering = la_neighborhoods_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(la_neighborhoods_grouped_clustering)

kmeans.labels_[0:10]

array([3, 1, 3, 3, 3, 3, 3, 3, 3, 0], dtype=int32)

#### Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [39]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

la_neighborhoods_merged = la_neighborhoods

la_neighborhoods_merged = la_neighborhoods_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

la_neighborhoods_merged.head() 

,Median Rent Price,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
120,1348.0,Long Beach,33.822661,-118.176187,3.0,Park,Clothing Store,Italian Restaurant,Boat or Ferry,Gas Station,Gym / Fitness Center,Seafood Restaurant,Construction & Landscaping,Gay Bar,Steakhouse
128,1330.0,Lancaster,34.711343,-118.157119,2.0,Trail,Yoga Studio,Event Space,Furniture / Home Store,French Restaurant,Football Stadium,Food Truck,Food & Drink Shop,Food,Flea Market
183,1333.0,Quartz Hill,34.638321,-118.221842,3.0,Print Shop,Jazz Club,Furniture / Home Store,French Restaurant,Football Stadium,Food Truck,Food & Drink Shop,Food,Flea Market,Fish & Chips Shop
223,1336.0,Torrance,33.877806,-118.330794,3.0,Park,Clothing Store,Mobile Phone Shop,Japanese Restaurant,Ice Cream Shop,Seafood Restaurant,Gift Shop,Grocery Store,Smoke Shop,Udon Restaurant
243,1343.0,Sun Valley,34.211160,-118.371144,3.0,Intersection,Furniture / Home Store,Construction & Landscaping,Gas Station,Storage Facility,Burger Joint,Grocery Store,Office,Home Service,Bus Station


#### Creatina a map of resulting clusters

In [41]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(la_neighborhoods_merged['Latitude'], la_neighborhoods_merged['Longitude'], la_neighborhoods_merged['Neighborhood'], la_neighborhoods_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='#3186cc',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Clusters Examination

### Cluster 1

In [44]:
la_neighborhoods_merged.loc[la_neighborhoods_merged['Cluster Labels'] == 0, la_neighborhoods_merged.columns[[1] + list(range(5, la_neighborhoods_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3775,Larchmont,Speakeasy,Gym,BBQ Joint,Gas Station,Garden,Furniture / Home Store,French Restaurant,Football Stadium,Food Truck,Food & Drink Shop
6926,Brentwood,Speakeasy,Cuban Restaurant,Garden,Furniture / Home Store,French Restaurant,Football Stadium,Food Truck,Food & Drink Shop,Food,Flea Market
13497,Hollywood Hills,BBQ Joint,Yoga Studio,Gas Station,Furniture / Home Store,French Restaurant,Football Stadium,Food Truck,Food & Drink Shop,Food,Flea Market
14106,Hollywood Hills,BBQ Joint,Yoga Studio,Gas Station,Furniture / Home Store,French Restaurant,Football Stadium,Food Truck,Food & Drink Shop,Food,Flea Market


### Cluster 2

In [45]:
la_neighborhoods_merged.loc[la_neighborhoods_merged['Cluster Labels'] == 1, la_neighborhoods_merged.columns[[1] + list(range(5, la_neighborhoods_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13038,Altadena,Intersection,Yoga Studio,Event Space,Furniture / Home Store,French Restaurant,Football Stadium,Food Truck,Food & Drink Shop,Food,Flea Market
16022,Wilmington,Baseball Field,Intersection,Yoga Studio,Event Space,Furniture / Home Store,French Restaurant,Football Stadium,Food Truck,Food & Drink Shop,Food
16183,Hawthorne,Intersection,Baseball Field,Park,Yoga Studio,Farm,Furniture / Home Store,French Restaurant,Football Stadium,Food Truck,Food & Drink Shop


### Cluster 3

In [46]:
la_neighborhoods_merged.loc[la_neighborhoods_merged['Cluster Labels'] == 2, la_neighborhoods_merged.columns[[1] + list(range(5, la_neighborhoods_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
128,Lancaster,Trail,Yoga Studio,Event Space,Furniture / Home Store,French Restaurant,Football Stadium,Food Truck,Food & Drink Shop,Food,Flea Market
981,Echo Park,Trail,Yoga Studio,Event Space,Furniture / Home Store,French Restaurant,Football Stadium,Food Truck,Food & Drink Shop,Food,Flea Market
6590,Echo Park,Trail,Yoga Studio,Event Space,Furniture / Home Store,French Restaurant,Football Stadium,Food Truck,Food & Drink Shop,Food,Flea Market
8243,Ladera Heights,Trail,Yoga Studio,Event Space,Furniture / Home Store,French Restaurant,Football Stadium,Food Truck,Food & Drink Shop,Food,Flea Market
8729,Lancaster,Trail,Yoga Studio,Event Space,Furniture / Home Store,French Restaurant,Football Stadium,Food Truck,Food & Drink Shop,Food,Flea Market
9941,Echo Park,Trail,Yoga Studio,Event Space,Furniture / Home Store,French Restaurant,Football Stadium,Food Truck,Food & Drink Shop,Food,Flea Market
11388,Lancaster,Trail,Yoga Studio,Event Space,Furniture / Home Store,French Restaurant,Football Stadium,Food Truck,Food & Drink Shop,Food,Flea Market
12753,Lancaster,Trail,Yoga Studio,Event Space,Furniture / Home Store,French Restaurant,Football Stadium,Food Truck,Food & Drink Shop,Food,Flea Market
12938,Echo Park,Trail,Yoga Studio,Event Space,Furniture / Home Store,French Restaurant,Football Stadium,Food Truck,Food & Drink Shop,Food,Flea Market
13347,Lancaster,Trail,Yoga Studio,Event Space,Furniture / Home Store,French Restaurant,Football Stadium,Food Truck,Food & Drink Shop,Food,Flea Market


### Cluster 4

In [47]:
la_neighborhoods_merged.loc[la_neighborhoods_merged['Cluster Labels'] == 3, la_neighborhoods_merged.columns[[1] + list(range(5, la_neighborhoods_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
120,Long Beach,Park,Clothing Store,Italian Restaurant,Boat or Ferry,Gas Station,Gym / Fitness Center,Seafood Restaurant,Construction & Landscaping,Gay Bar,Steakhouse
183,Quartz Hill,Print Shop,Jazz Club,Furniture / Home Store,French Restaurant,Football Stadium,Food Truck,Food & Drink Shop,Food,Flea Market,Fish & Chips Shop
223,Torrance,Park,Clothing Store,Mobile Phone Shop,Japanese Restaurant,Ice Cream Shop,Seafood Restaurant,Gift Shop,Grocery Store,Smoke Shop,Udon Restaurant
243,Sun Valley,Intersection,Furniture / Home Store,Construction & Landscaping,Gas Station,Storage Facility,Burger Joint,Grocery Store,Office,Home Service,Bus Station
254,Pacoima,Supermarket,Chinese Restaurant,Snack Place,Dessert Shop,Restaurant,Food Truck,Big Box Store,Coffee Shop,Bank,Clothing Store
265,Baldwin Park,Latin American Restaurant,Coffee Shop,Theater,Mexican Restaurant,Chinese Restaurant,Clothing Store,Fast Food Restaurant,Fish & Chips Shop,Yoga Studio,Farm
296,Artesia,Indian Restaurant,Bubble Tea Shop,Playground,Chinese Restaurant,Ice Cream Shop,Vietnamese Restaurant,Sandwich Place,Dessert Shop,Sushi Restaurant,Bakery
321,Long Beach,Park,Clothing Store,Italian Restaurant,Boat or Ferry,Gas Station,Gym / Fitness Center,Seafood Restaurant,Construction & Landscaping,Gay Bar,Steakhouse
329,Mid-Wilshire,Gym,Museum,Park,Furniture / Home Store,Bus Stop,Big Box Store,Fast Food Restaurant,Salad Place,Beach,Mexican Restaurant
371,Pacoima,Supermarket,Chinese Restaurant,Snack Place,Dessert Shop,Restaurant,Food Truck,Big Box Store,Coffee Shop,Bank,Clothing Store


### Cluster 5

In [48]:
la_neighborhoods_merged.loc[la_neighborhoods_merged['Cluster Labels'] == 4, la_neighborhoods_merged.columns[[1] + list(range(5, la_neighborhoods_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
863,Compton,Construction & Landscaping,Yoga Studio,Event Space,Furniture / Home Store,French Restaurant,Football Stadium,Food Truck,Food & Drink Shop,Food,Flea Market
5383,Granada Hills,Construction & Landscaping,Food Truck,Yoga Studio,Event Space,Furniture / Home Store,French Restaurant,Football Stadium,Food & Drink Shop,Food,Flea Market
7703,Granada Hills,Construction & Landscaping,Food Truck,Yoga Studio,Event Space,Furniture / Home Store,French Restaurant,Football Stadium,Food & Drink Shop,Food,Flea Market
15935,San Fernando,Construction & Landscaping,Yoga Studio,Event Space,Furniture / Home Store,French Restaurant,Football Stadium,Food Truck,Food & Drink Shop,Food,Flea Market


# Result  and Discussion section

We examined this dataset using two main perspectives.
The first is finding neighborhoods that average rent price fit Median Rent Price among all Neighborhoods.
The second is clustering found neighborhoods using data about venues located around them.
 
As a result, we can see that the Neighborhoods from Cluster 4 can be considered more suitable for newcomers to rent apartments, because they contain a big variety of different venues in close range which can satisfy their needs. Other Clusters are not so diversified and can be suitable for immigrants who have enough experience of living in Los Angeles and familiar with the city. 


# Conclusion

According to the Migration Policy Institute, more than 14% of the population of the United States were considered immigrants in 2018. From which we can assume that immigrants have become a major part of the US society and one of the drivers of cultural changes. In this context of a continuous influx of migrants it is urgent to adopt machine learning tools in order to assist newcomers to find a suitable place to settle down. As a result, the question we were trying to solve was: how could we provide support to immigrants in finding suitable apartments in example of Los Angeles? 

To solve this problem, we clustered LA neighborhoods in order to recommend neighborhoods with the median renting price amount where immigrants can find suitable apartments. We recommended neighborhoods according to essential facilities and services surrounding such venues i.e. schools, stores, restaurants, hospitals, and entertainments.

We gathered LA Neighborhoods and the Median Rent Price data from USC Price Center for Social Innovation (https://usc.data.socrata.com/Los-Angeles/Rent-Price-LA-/4a97-v5tx). Moreover, to explore and target recommended locations across different venues according to the presence of essential facilities and services we accessed data through the FourSquare API interface and arranged them as a data frame for visualization. By merging LA Neighborhoods and the Median Rent Price data and data on essential facilities and services surrounding such properties from the FourSquare API interface, we were able to recommend suitable neighborhoods to rent apartments.

The Methodology section comprised four stages: 1. Data Collection; 2. Data Exploration and Understanding; 3. Data preparation and preprocessing; 4. Modeling. In the modeling section, we used the k-means clustering technique as it is highly flexible to account for changes in the median rent price amount in Los Angeles Neighborhoods.

Finally, we drew the conclusion suggesting suitable neighborhoods to rent apartments. We used two main perspectives to reach our results. First, we examined them according to median price amount paid around all neighborhoods. Second, we analyzed our results according to the five clusters we produced and found that Cluster 4 might be considered as the most suitable.